In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
circuits = pd.read_csv('../data/circuits.csv')
constructor_results = pd.read_csv('../data/constructor_results.csv')
constructor_standings = pd.read_csv('../data/constructor_standings.csv')
constructor = pd.read_csv('../data/constructors.csv')
drivers = pd.read_csv('../data/drivers.csv')
driver_standings = pd.read_csv('../data/driver_standings.csv')
lap_times = pd.read_csv('../data/lap_times.csv')
pit_stops = pd.read_csv('../data/pit_stops.csv')
qualif = pd.read_csv('../data/qualifying.csv')
races = pd.read_csv('../data/races.csv')
results = pd.read_csv('../data/results.csv')
seasons = pd.read_csv('../data/seasons.csv')
status = pd.read_csv('../data/status.csv')

NameError: name 'circuit' is not defined